<p style="font-family:Times;font-size:175%; text-align:center"> <font color='blue'>Back Testing of Simple Moving Average Indicator</font></p>

___
> **Input** :
>* Simulation time (the length of the simulation back testing will run in days)
>* SMA (simple moving average indicator) 

> **Procedure** :
>* If yesterday's closing price was on the line or below the SMA line, today we will open a long position
>* If yesterday's closing price was above the SMA line, today we will open a short position
>* The position will be closed when today's price closes

> **Output** :
>* Profit on long positions
>* Profit on short positions
>* Total profit

>* Loss on long positions
>* Loss on short positions
>* Total loss

>* Winning rate
>* Return on Investment

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf #yahoo finance 
import matplotlib.pyplot as plt

In [2]:
#input
capital = 100 #($)
simulation_time = 1000 #days
SMA = 30 #simple moving average
pair = 'BTC-USD'
location = 'C:/Users/Musadadhudori/Music/CRYTHOCURRENCY/Robot Trading/BTC/Single Target/Portfolio/Journal numpy.xlsx'

In [3]:
start_date = pd.Timestamp.today() - pd.Timedelta(days=simulation_time + SMA)
end_date = pd.Timestamp.today()
data = yf.download(pair, start =start_date)
start_date += pd.Timedelta(days=SMA)

[*********************100%***********************]  1 of 1 completed


In [4]:
print('Simulation runs from ' + '\033[1m' + str(start_date) + '\033[0m' + ' to ' + '\033[1m' + str(end_date) + '\033[0m')

c = np.flip(np.array(data.Close))
o = np.flip(np.array(data.Close.shift(1)))

arr_win_long  = (c[1:simulation_time+1]>=[np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*((c/o)[:simulation_time]>1)*1
arr_win_short = (c[1:simulation_time+1]< [np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*((c/o)[:simulation_time]<1)*1
print('Winrate Method: %5.2f' %(100*(np.sum(arr_win_long + arr_win_short))/simulation_time) + '%\n')

print('\033[1m' + 'Simple Interest:'+ '\033[0m')
print('Return if hold and fix capital ($): %5.2f' %(capital*sum((c/o)[:simulation_time]-1)) +'\n')

arr_long_simp  = (c[1:simulation_time+1]>=[np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*((c/o)[:simulation_time]-1)
arr_short_simp = (c[1:simulation_time+1]< [np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*(1-(c/o)[:simulation_time])

profit_long = np.sum(arr_long_simp, where = arr_long_simp>0)
loss_long   = np.sum(arr_long_simp, where = arr_long_simp<0)
#print('profit from Long Positions ($): %5.2f' %(capital*(profit_long)) )
#print('loss   from Long Positions ($): %5.2f' %(capital*(loss_long)) )
print('return from Long Positions ($): %5.2f' %(capital*(profit_long + loss_long)) + '\n')

profit_short = np.sum(arr_short_simp, where = arr_short_simp>0)
loss_short   = np.sum(arr_short_simp, where = arr_short_simp<0)
#print('profit from Short Positions ($): %5.2f' %(capital*(profit_short)) )
#print('loss   from Short Positions ($): %5.2f' %(capital*(loss_short)) )
print('return from Short Positions ($): %5.2f' %(capital*(profit_short + loss_short)) + '\n')

print('Return Method ($): %5.2f' %(capital*(profit_long + profit_short + loss_long + loss_short)) + '\n\n')


print('\033[1m' + 'Compounded Interest:'+ '\033[0m')
print('Return Hold ($): %5.2f' %(capital*(c[0]/o[simulation_time-1] - 1)) +'\n')

arr_long_comp  = (c[1:simulation_time+1]>=[np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*((c/o)[:simulation_time])
arr_short_comp = (c[1:simulation_time+1]< [np.mean(c[i: i+SMA]) for i in range(1, simulation_time+1)])*(2-(c/o)[:simulation_time])

profit_long = np.prod(arr_long_comp, where = arr_long_comp>1)
loss_long   = np.prod(arr_long_comp, where = arr_long_comp>0)/profit_long
#print('profit from Long Positions ($): %5.2f' %(capital*(profit_long - 1)) )
#print('loss   from Long Positions ($): %5.2f' %(capital*(loss_long - 1)) )
print('return from Long Positions ($): %5.2f' %(capital*(profit_long*loss_long - 1)) + '\n')

profit_short = np.prod(arr_short_comp, where = arr_short_comp>1)
loss_short   = np.prod(arr_short_comp, where = arr_short_comp>0)/profit_short
#print('profit from Short Positions ($): %5.2f' %(capital*(profit_short-1)) )
#print('loss   from Short Positions ($): %5.2f' %(capital*(loss_short-1)) )
print('return from Short Positions ($): %5.2f' %(capital*(profit_short*loss_short-1)) + '\n')

print('Return Method ($): %5.2f' %(capital*(profit_long*profit_short*loss_long*loss_short-1)) )

Simulation runs from 2020-01-23 09:16:44.613662 to 2022-10-19 09:16:44.613662
Winrate Method: 50.40%

Simple Interest:
Return if hold and fix capital ($): 156.85

return from Long Positions ($): 175.74

return from Short Positions ($): 18.89

Return Method ($): 194.63


Compounded Interest:
Return Hold ($): 122.72

return from Long Positions ($): 321.51

return from Short Positions ($): -20.34

Return Method ($): 235.79


In [5]:
journal = pd.DataFrame()

arr_date =  np.array([start_date + pd.Timedelta(days=i) for i in range(simulation_time)])

typ = np.where(arr_long_comp>0, 'long', 'short')
typ = np.flip(typ)

arr_win = arr_win_short + arr_win_long
arr_win = np.flip(arr_win)
arr_winrate = np.cumsum(arr_win)/np.arange(1, simulation_time+1)

arr_comp = arr_long_comp + arr_short_comp
arr_comp = np.flip(arr_comp)

arr_simp = arr_long_simp + arr_short_simp
arr_simp = np.flip(arr_simp)

arr_long_comp = np.where(arr_long_comp>0, arr_long_comp, 1)
arr_short_comp = np.where(arr_short_comp>0, arr_short_comp, 1)

journal['Date'] = arr_date
journal['Type'] = typ
journal['Winrate'] = arr_winrate

journal['Hold simple'] = capital*(np.cumsum(np.flip((c/o)[:simulation_time]-1))+1)
journal['Simple long'] = capital*(np.cumsum(np.flip(arr_long_simp)) + 1)
journal['Simple short'] = capital*(np.cumsum(np.flip(arr_short_simp)) + 1)
journal['Return simple'] = capital*(np.cumsum(arr_simp) + 1)

journal['Hold compounded'] = capital*np.array([c[i]/o[simulation_time-1] for i in range(simulation_time-1, -1,-1)])
journal['Compounded long'] = capital*np.cumprod(np.flip(arr_long_comp))
journal['Compounded short'] = capital*np.cumprod(np.flip(arr_short_comp))
journal['Return compounded'] = capital*np.cumprod(arr_comp)

journal.to_excel(location, index = False)